Whole 3D volume

In [29]:
import os
import csv
import nibabel as nib
import numpy as np
from scipy.fftpack import dctn
import sys

# === Configuration ===
subjects = ["sub-001", "sub-002", "sub-003"]
positions = ["up", "down", "left", "right"]
recons = ["full", "50p", "75p", "95p"]  # woBin handled separately

base_dir = "/home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions"

# === Safe print (prevents duplicate display in Jupyter/VSCode) ===
def log(message):
    sys.stdout.write(message + "\n")
    sys.stdout.flush()

# === Helper: load and normalize MRI volumes ===
def load_nifti_as_array(path):
    img = nib.load(path)
    data = img.get_fdata()
    # Normalize to [0,1]
    data = (data - np.min(data)) / (np.max(data) - np.min(data) + 1e-8)
    return data

# === Compute DCTE for 3D volume ===
def compute_dcte_3d(volume, hf_ratio=0.5):
    coeffs = dctn(volume, norm='ortho')
    abs_coeffs = np.abs(coeffs)

    nx, ny, nz = abs_coeffs.shape
    cx, cy, cz = int(nx * (1 - hf_ratio)), int(ny * (1 - hf_ratio)), int(nz * (1 - hf_ratio))

    low_freq = abs_coeffs[:cx, :cy, :cz].mean()
    high_freq = abs_coeffs[cx:, cy:, cz:].mean()

    mean_intensity = np.mean(volume) + 1e-8
    return (high_freq / (low_freq + 1e-8)) / mean_intensity

# === Optional: slice-wise DCTE ===
def compute_dcte_slicewise(volume, hf_ratio=0.5):
    """Compute DCTE per slice (along z-axis) and return the mean."""
    dcte_vals = []
    for z in range(volume.shape[2]):
        slice_ = volume[:, :, z]
        coeffs = dctn(slice_, norm='ortho')
        abs_coeffs = np.abs(coeffs)

        nx, ny = abs_coeffs.shape
        cutoff_x = int(nx * (1 - hf_ratio))
        cutoff_y = int(ny * (1 - hf_ratio))

        low_freq_energy = np.mean(abs_coeffs[:cutoff_x, :cutoff_y])
        high_freq_energy = np.mean(abs_coeffs[cutoff_x:, cutoff_y:])
        mean_intensity = np.mean(slice_) + 1e-8

        dcte_vals.append((high_freq_energy / (low_freq_energy + 1e-8)) / mean_intensity)
    return np.mean(dcte_vals)

# === Main computation loop ===
results = []
use_slice_mode = False  # True = compute per-slice DCTE, then average

log("=== Starting DCTE computation ===")

for sub in subjects:
    # --- Compute woBin (only once per subject) ---
    woBin_path = os.path.join(base_dir, sub, "anat", f"{sub}_rec-stevaFull_T1w.nii.gz")
    if not os.path.exists(woBin_path):
        log(f"⚠️ Missing woBin file for {sub}: {woBin_path}")
        dcte_value = np.nan
    else:
        vol = load_nifti_as_array(woBin_path)
        dcte_value = compute_dcte_slicewise(vol) if use_slice_mode else compute_dcte_3d(vol)
        log(f"✅ {sub} | woBin: DCTE = {dcte_value:.4e}")
    # record region explicitly so downstream merging is consistent
    results.append((sub, "none", "woBin", "WholeVolume", dcte_value))

    # --- Compute for each gaze position and undersampling level ---
    for pos in positions:
        for level in recons:
            tag = "stevafull" if level == "full" else f"steva{level}"
            img_path = os.path.join(base_dir, sub, "anat", f"{sub}_acq-{pos}_rec-{tag}_T1w.nii.gz")

            if not os.path.exists(img_path):
                log(f"⚠️ Missing file for {sub} {pos} {level}")
                dcte_value = np.nan
            else:
                vol = load_nifti_as_array(img_path)
                dcte_value = compute_dcte_slicewise(vol) if use_slice_mode else compute_dcte_3d(vol)
                log(f"✅ {sub} | {pos} | {level}: DCTE = {dcte_value:.4e}")

            # include region for consistency with brain/eyes CSVs
            results.append((sub, pos, level, "WholeVolume", dcte_value))

# === Save results ===
out_csv = "dcte_volume.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "position", "reconstruction", "region", "dcte"])
    writer.writerows(results)

log(f"\n📁 DCTE results saved to: {out_csv}")


=== Starting DCTE computation ===
✅ sub-001 | woBin: DCTE = 9.6723e-01
✅ sub-001 | up | full: DCTE = 1.2547e+00
✅ sub-001 | up | 50p: DCTE = 1.2746e+00
✅ sub-001 | up | 75p: DCTE = 1.0865e+00
✅ sub-001 | up | 95p: DCTE = 8.6158e-01
✅ sub-001 | down | full: DCTE = 1.2793e+00
✅ sub-001 | down | 50p: DCTE = 1.2336e+00
✅ sub-001 | down | 75p: DCTE = 1.2089e+00
✅ sub-001 | down | 95p: DCTE = 8.2896e-01
✅ sub-001 | left | full: DCTE = 1.2712e+00
✅ sub-001 | left | 50p: DCTE = 1.2761e+00
✅ sub-001 | left | 75p: DCTE = 1.3086e+00
✅ sub-001 | left | 95p: DCTE = 9.2289e-01
✅ sub-001 | right | full: DCTE = 1.2732e+00
✅ sub-001 | right | 50p: DCTE = 1.3021e+00
✅ sub-001 | right | 75p: DCTE = 1.1148e+00
✅ sub-001 | right | 95p: DCTE = 8.1887e-01
✅ sub-002 | woBin: DCTE = 1.1711e+00
✅ sub-002 | up | full: DCTE = 1.4486e+00
✅ sub-002 | up | 50p: DCTE = 1.2841e+00
✅ sub-002 | up | 75p: DCTE = 9.9116e-01
✅ sub-002 | up | 95p: DCTE = 8.3030e-01
✅ sub-002 | down | full: DCTE = 1.5321e+00
✅ sub-002 | down

Brain mask - filename mapping

In [30]:
import os
import glob
import csv

# Path to MRIQC anat work directory
work_dir = "/tmp/mriqc/work/mriqc_wf/anatMRIQC"

# Prepare list for results
mapping = []

# Loop over hash directories
for subdir in sorted(os.listdir(work_dir)):
    hash_dir = os.path.join(work_dir, subdir)
    if not os.path.isdir(hash_dir):
        continue

    # Look for a *_conformed.nii.gz file inside any subfolder
    conformed_files = glob.glob(os.path.join(hash_dir, "**", "*_conformed.nii.gz"), recursive=True)
    if not conformed_files:
        continue

    # Take the first (there should be only one per folder)
    conf_path = conformed_files[0]
    base = os.path.basename(conf_path)
    original_name = base.replace("_conformed.nii.gz", "")  # remove suffix

    mapping.append((original_name, subdir))

# Save to CSV
out_csv = "mriqc_filename_mapping.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["original_filename", "encoded_id"])
    writer.writerows(mapping)

print(f"✅ Mapping saved to: {out_csv}")
print(f"🔢 Found {len(mapping)} entries")

# Example of access:
# dict_mapping = {orig: enc for orig, enc in mapping}


✅ Mapping saved to: mriqc_filename_mapping.csv
🔢 Found 51 entries


Brain mask - DCTE

In [31]:
import os
import csv
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to
from scipy.fftpack import dctn

# === Configuration ===
subjects = ["sub-001", "sub-002", "sub-003"]
positions = ["up", "down", "left", "right"]
recon_levels = ["woBin", "full", "50p", "75p", "95p"]

base_dir = "/home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions"
mriqc_work_dir = "/tmp/mriqc/work/mriqc_wf/anatMRIQC"
mapping_csv = "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/mriqc_filename_mapping.csv"

# === Helper functions ===

def load_mapping(csv_path):
    """Load mapping from original filename to encoded ID."""
    mapping = {}
    with open(csv_path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            mapping[row["original_filename"]] = row["encoded_id"]
    return mapping


def load_nifti_as_array(path):
    img = nib.load(path)
    data = img.get_fdata()
    # Normalize to [0, 1]
    data = (data - np.min(data)) / (np.max(data) - np.min(data) + 1e-8)
    return img, data


def compute_dcte_3d(volume, hf_ratio=0.5):
    coeffs = dctn(volume, norm='ortho')
    abs_coeffs = np.abs(coeffs)

    nx, ny, nz = abs_coeffs.shape
    cx, cy, cz = int(nx * (1 - hf_ratio)), int(ny * (1 - hf_ratio)), int(nz * (1 - hf_ratio))

    low_freq = abs_coeffs[:cx, :cy, :cz].mean()
    high_freq = abs_coeffs[cx:, cy:, cz:].mean()

    mean_intensity = np.mean(volume) + 1e-8
    return (high_freq / (low_freq + 1e-8)) / mean_intensity


def load_brain_mask(sub, pos, mapping):
    """Load segmentation (CSF/GM/WM) for the reference (stevafull) image."""
    orig_name = f"{sub}_acq-{pos}_rec-stevafull_T1w"
    if orig_name not in mapping:
        print(f"⚠️ No mapping found for {orig_name}")
        return None, None

    encoded = mapping[orig_name]
    mask_path = os.path.join(
        mriqc_work_dir,
        "brain_tissue_segmentation",
        encoded,
        "segmentation",
        "segment.nii.gz"
    )

    if not os.path.exists(mask_path):
        print(f"⚠️ Mask not found for {orig_name}")
        return None, None

    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    return mask_img, mask_data


# === Main computation ===
mapping = load_mapping(mapping_csv)
results = []

print("=== Starting DCTE computation (tissue-based) ===")

for sub in subjects:
    # Load woBin once per subject (woBin is independent of gaze position)
    woBin_path = os.path.join(base_dir, sub, "anat", f"{sub}_rec-stevaFull_T1w.nii.gz")
    if os.path.exists(woBin_path):
        wo_img, wo_data = load_nifti_as_array(woBin_path)
        wo_exists = True
    else:
        wo_img = None
        wo_data = None
        wo_exists = False

    # If woBin exists, compute tissue-wise DCTE once and store as a single row per subject
    if wo_exists:
        # We need a reference mask to compute tissues; try any available position mask
        ref_mask_found = False
        for try_pos in positions:
            mask_img_try, mask_data_try = load_brain_mask(sub, try_pos, mapping)
            if mask_data_try is not None:
                # Resample mask to woBin space if needed
                if mask_img_try.shape != wo_img.shape:
                    mask_resampled_wobin = resample_from_to(mask_img_try, wo_img, order=0).get_fdata()
                else:
                    mask_resampled_wobin = mask_data_try

                tissue_classes_wobin = {
                    "CSF": mask_resampled_wobin == 1,
                    "GM": mask_resampled_wobin == 2,
                    "WM": mask_resampled_wobin == 3,
                    "Brain": np.isin(mask_resampled_wobin, [1, 2, 3])
                }

                for tissue_name, tissue_mask in tissue_classes_wobin.items():
                    if np.sum(tissue_mask) == 0:
                        dcte_val = np.nan
                    else:
                        vol_masked = np.copy(wo_data)
                        vol_masked[~tissue_mask] = 0
                        dcte_val = compute_dcte_3d(vol_masked)

                    # Use position 'none' for subject-level woBin (independent of gaze)
                    results.append((sub, "none", "woBin", tissue_name, dcte_val))
                    print(f"✅ {sub} | woBin | {tissue_name}: DCTE = {dcte_val:.4e}")

                ref_mask_found = True
                break

        if not ref_mask_found:
            # Could not find any mask for this subject to compute woBin tissue DCTE
            tissue_classes_missing = ["CSF", "GM", "WM", "Brain"]
            for tissue_name in tissue_classes_missing:
                results.append((sub, "none", "woBin", tissue_name, np.nan))
                print(f"⚠️ {sub} | woBin present but no mask found -> {tissue_name}: DCTE = nan")
    else:
        # woBin missing: record NaNs once per subject
        tissue_classes_missing = ["CSF", "GM", "WM", "Brain"]
        for tissue_name in tissue_classes_missing:
            results.append((sub, "none", "woBin", tissue_name, np.nan))
            print(f"⚠️ {sub} | woBin missing -> {tissue_name}: DCTE = nan")

    # Now compute position-dependent reconstructions
    for pos in positions:
        # Load mask once per position
        mask_img, mask_data = load_brain_mask(sub, pos, mapping)
        if mask_data is None:
            continue

        for level in ["full", "50p", "75p", "95p"]:
            tag = "stevafull" if level == "full" else f"steva{level}"
            img_path = os.path.join(base_dir, sub, "anat", f"{sub}_acq-{pos}_rec-{tag}_T1w.nii.gz")

            if not os.path.exists(img_path):
                print(f"⚠️ Missing file for {sub} {pos} {level}")
                # Append NaNs for all tissues for this missing reconstruction (exclude WholeVolume)
                for tissue_name in ["CSF", "GM", "WM", "Brain"]:
                    results.append((sub, pos, level, tissue_name, np.nan))
                continue

            img, data = load_nifti_as_array(img_path)

            # Resample mask if needed (to match this reconstruction's image)
            if mask_img.shape != img.shape:
                mask_resampled = resample_from_to(mask_img, img, order=0).get_fdata()
            else:
                mask_resampled = mask_data

            # Compute for each tissue (exclude WholeVolume)
            tissue_classes = {
                "CSF": mask_resampled == 1,
                "GM": mask_resampled == 2,
                "WM": mask_resampled == 3,
                "Brain": np.isin(mask_resampled, [1, 2, 3])
            }

            for tissue_name, tissue_mask in tissue_classes.items():
                if np.sum(tissue_mask) == 0:
                    dcte_val = np.nan
                else:
                    vol_masked = np.copy(data)
                    vol_masked[~tissue_mask] = 0
                    dcte_val = compute_dcte_3d(vol_masked)

                results.append((sub, pos, level, tissue_name, dcte_val))
                print(f"✅ {sub} | {pos} | {level} | {tissue_name}: DCTE = {dcte_val:.4e}")

# === Save results ===
out_csv = "dcte_brain.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "position", "reconstruction", "tissue", "dcte"])
    writer.writerows(results)

print(f"\n📁 Tissue-wise DCTE results saved to: {out_csv}")


=== Starting DCTE computation (tissue-based) ===
✅ sub-001 | woBin | CSF: DCTE = 4.9725e+01
✅ sub-001 | woBin | GM: DCTE = 2.7304e+01
✅ sub-001 | woBin | WM: DCTE = 1.1712e+01
✅ sub-001 | woBin | Brain: DCTE = 2.1498e+00
✅ sub-001 | up | full | CSF: DCTE = 4.9114e+01
✅ sub-001 | up | full | GM: DCTE = 2.7109e+01
✅ sub-001 | up | full | WM: DCTE = 1.1597e+01
✅ sub-001 | up | full | Brain: DCTE = 2.4886e+00
✅ sub-001 | up | 50p | CSF: DCTE = 5.2298e+01
✅ sub-001 | up | 50p | GM: DCTE = 2.8634e+01
✅ sub-001 | up | 50p | WM: DCTE = 1.2246e+01
✅ sub-001 | up | 50p | Brain: DCTE = 2.5340e+00
✅ sub-001 | up | 75p | CSF: DCTE = 4.9366e+01
✅ sub-001 | up | 75p | GM: DCTE = 2.7091e+01
✅ sub-001 | up | 75p | WM: DCTE = 1.1603e+01
✅ sub-001 | up | 75p | Brain: DCTE = 2.2827e+00
✅ sub-001 | up | 95p | CSF: DCTE = 4.6489e+01
✅ sub-001 | up | 95p | GM: DCTE = 2.6408e+01
✅ sub-001 | up | 95p | WM: DCTE = 1.1374e+01
✅ sub-001 | up | 95p | Brain: DCTE = 2.1730e+00
✅ sub-001 | down | full | CSF: DCTE = 4

Eyes mask - filename mapping

In [32]:
import os
import glob
import csv

# Path to MRIQC anat work directory
work_dir = "/tmp/mriqc/work_eyes/mriqc_wf/anatMRIQC"

# Prepare list for results
mapping = []

# Loop over hash directories
for subdir in sorted(os.listdir(work_dir)):
    hash_dir = os.path.join(work_dir, subdir)
    if not os.path.isdir(hash_dir):
        continue

    # Look for a *_conformed.nii.gz file inside any subfolder
    conformed_files = glob.glob(os.path.join(hash_dir, "**", "*_conformed.nii.gz"), recursive=True)
    if not conformed_files:
        continue

    # Take the first (there should be only one per folder)
    conf_path = conformed_files[0]
    base = os.path.basename(conf_path)
    original_name = base.replace("_conformed.nii.gz", "")  # remove suffix

    mapping.append((original_name, subdir))

# Save to CSV
out_csv = "mriqc_eyes_filename_mapping.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["original_filename", "encoded_id"])
    writer.writerows(mapping)

print(f"✅ Mapping saved to: {out_csv}")
print(f"🔢 Found {len(mapping)} entries")

# Example of access:
# dict_mapping = {orig: enc for orig, enc in mapping}


✅ Mapping saved to: mriqc_eyes_filename_mapping.csv
🔢 Found 51 entries


Eyes mask - DCTE

In [33]:
import os
import csv
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to
from scipy.fftpack import dctn

# === Configuration ===
subjects = ["sub-001", "sub-002", "sub-003"]
positions = ["up", "down", "left", "right"]
recon_levels = ["woBin", "full", "50p", "75p", "95p"]

base_dir = "/home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions"
mriqc_work_dir = "/tmp/mriqc/work_eyes/mriqc_wf/anatMRIQC/SpatialNormalization"
mapping_csv = "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/mriqc_eyes_filename_mapping.csv"

# === Helper functions ===

def load_mapping(csv_path):
    """Load mapping from original filename to encoded ID."""
    mapping = {}
    with open(csv_path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            mapping[row["original_filename"]] = row["encoded_id"]
    return mapping


def load_nifti_as_array(path):
    img = nib.load(path)
    data = img.get_fdata()
    # Normalize to [0,1]
    data = (data - np.min(data)) / (np.max(data) - np.min(data) + 1e-8)
    return img, data


def compute_dcte_3d(volume, hf_ratio=0.5):
    coeffs = dctn(volume, norm='ortho')
    abs_coeffs = np.abs(coeffs)

    nx, ny, nz = abs_coeffs.shape
    cx, cy, cz = int(nx * (1 - hf_ratio)), int(ny * (1 - hf_ratio)), int(nz * (1 - hf_ratio))

    low_freq = abs_coeffs[:cx, :cy, :cz].mean()
    high_freq = abs_coeffs[cx:, cy:, cz:].mean()

    mean_intensity = np.mean(volume) + 1e-8
    return (high_freq / (low_freq + 1e-8)) / mean_intensity


def load_eyes_mask(sub, pos, mapping):
    """Load the eyes mask (binary) for the reference (stevafull) image."""
    orig_name = f"{sub}_acq-{pos}_rec-stevafull_T1w"
    if orig_name not in mapping:
        print(f"⚠️ No mapping found for {orig_name}")
        return None, None

    encoded = mapping[orig_name]
    mask_path = os.path.join(
        mriqc_work_dir,
        encoded,
        "eyes_std2t1w",
        "tpl-MNI152NLin2009cAsym_res-01_desc-eye_mask_trans.nii.gz"
    )

    if not os.path.exists(mask_path):
        print(f"⚠️ Eyes mask not found for {orig_name}")
        return None, None

    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    return mask_img, mask_data


# === Main computation ===
mapping = load_mapping(mapping_csv)
results = []

print("=== Starting DCTE computation (eyes mask) ===")

for sub in subjects:
    # Load woBin image once per subject if available (woBin is not position-specific)
    woBin_path = os.path.join(base_dir, sub, "anat", f"{sub}_rec-stevaFull_T1w.nii.gz")
    if os.path.exists(woBin_path):
        wo_img, wo_data = load_nifti_as_array(woBin_path)
        wo_exists = True
    else:
        wo_img = None
        wo_data = None
        wo_exists = False

    # If woBin exists, compute eyes-region DCTE once and store as a single row per subject
    if wo_exists:
        ref_mask_found = False
        for try_pos in positions:
            mask_img_try, mask_data_try = load_eyes_mask(sub, try_pos, mapping)
            if mask_data_try is not None:
                # Resample eyes mask to woBin space if needed
                if mask_img_try.shape != wo_img.shape:
                    mask_resampled_wobin = resample_from_to(mask_img_try, wo_img, order=0).get_fdata()
                else:
                    mask_resampled_wobin = mask_data_try

                eyes_mask_wobin = mask_resampled_wobin > 0
                if np.sum(eyes_mask_wobin) == 0:
                    dcte_val = np.nan
                else:
                    vol_masked = np.copy(wo_data)
                    vol_masked[~eyes_mask_wobin] = 0
                    dcte_val = compute_dcte_3d(vol_masked)

                # Use position 'none' for subject-level woBin (independent of gaze)
                results.append((sub, "none", "woBin", "Eyes", dcte_val))
                print(f"✅ {sub} | woBin | Eyes: DCTE = {dcte_val:.4e}")

                ref_mask_found = True
                break

        if not ref_mask_found:
            results.append((sub, "none", "woBin", "Eyes", np.nan))
            print(f"⚠️ {sub} | woBin present but no eyes mask found -> Eyes: DCTE = nan")
    else:
        results.append((sub, "none", "woBin", "Eyes", np.nan))
        print(f"⚠️ {sub} | woBin missing -> Eyes: DCTE = nan")

    # For position-dependent reconstructions compute per-position Eyes DCTE
    for pos in positions:
        mask_img, mask_data = load_eyes_mask(sub, pos, mapping)
        if mask_data is None:
            continue

        for level in ["full", "50p", "75p", "95p"]:
            tag = "stevafull" if level == "full" else f"steva{level}"
            img_path = os.path.join(base_dir, sub, "anat", f"{sub}_acq-{pos}_rec-{tag}_T1w.nii.gz")

            if not os.path.exists(img_path):
                print(f"⚠️ Missing file for {sub} {pos} {level}")
                results.append((sub, pos, level, "Eyes", np.nan))
                continue

            img, data = load_nifti_as_array(img_path)

            # Resample mask if needed
            if mask_img.shape != img.shape:
                mask_resampled = resample_from_to(mask_img, img, order=0).get_fdata()
            else:
                mask_resampled = mask_data

            # Binary eyes mask
            eyes_mask = mask_resampled > 0

            if np.sum(eyes_mask) == 0:
                dcte_val = np.nan
            else:
                vol_masked = np.copy(data)
                vol_masked[~eyes_mask] = 0
                dcte_val = compute_dcte_3d(vol_masked)

            results.append((sub, pos, level, "Eyes", dcte_val))
            print(f"✅ {sub} | {pos} | {level} | Eyes: DCTE = {dcte_val:.4e}")

# === Save results ===
out_csv = "dcte_eyes.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "position", "reconstruction", "region", "dcte"])
    writer.writerows(results)

print(f"\n📁 Eyes-region DCTE results saved to: {out_csv}")


=== Starting DCTE computation (eyes mask) ===
✅ sub-001 | woBin | Eyes: DCTE = 2.4792e+01
✅ sub-001 | up | full | Eyes: DCTE = 2.7257e+01
✅ sub-001 | up | 50p | Eyes: DCTE = 2.8430e+01
✅ sub-001 | up | 75p | Eyes: DCTE = 2.5528e+01
✅ sub-001 | up | 95p | Eyes: DCTE = 2.4518e+01
✅ sub-001 | down | full | Eyes: DCTE = 2.8279e+01
✅ sub-001 | down | 50p | Eyes: DCTE = 2.7589e+01
✅ sub-001 | down | 75p | Eyes: DCTE = 2.8482e+01
✅ sub-001 | down | 95p | Eyes: DCTE = 2.2665e+01
✅ sub-001 | left | full | Eyes: DCTE = 2.8195e+01
✅ sub-001 | left | 50p | Eyes: DCTE = 2.8365e+01
✅ sub-001 | left | 75p | Eyes: DCTE = 3.0665e+01
✅ sub-001 | left | 95p | Eyes: DCTE = 2.5050e+01
✅ sub-001 | right | full | Eyes: DCTE = 2.8359e+01
✅ sub-001 | right | 50p | Eyes: DCTE = 2.9666e+01
✅ sub-001 | right | 75p | Eyes: DCTE = 2.7110e+01
✅ sub-001 | right | 95p | Eyes: DCTE = 2.3705e+01
✅ sub-002 | woBin | Eyes: DCTE = 2.4031e+01
✅ sub-002 | up | full | Eyes: DCTE = 2.6706e+01
✅ sub-002 | up | 50p | Eyes: DCTE 

Joint dataframe

In [3]:
import pandas as pd
import os

# === Input files ===
dcte_files = {
    "volume": "dcte_volume.csv",
    "brain": "dcte_brain.csv",
    "eyes": "dcte_eyes.csv"
}

dfs = []
for mask_type, path in dcte_files.items():
    if not os.path.exists(path):
        print(f"⚠️ Missing file: {path}")
        continue

    df = pd.read_csv(path)

    # Normalize column names to: subject, position, reconstruction, region, dcte
    if "tissue" in df.columns:
        df.rename(columns={"tissue": "region"}, inplace=True)
    if "mask" in df.columns and "region" not in df.columns:
        # some files may use 'mask' as region; prefer explicit 'region'
        df.rename(columns={"mask": "region"}, inplace=True)
    if "comparison" in df.columns and "reconstruction" not in df.columns:
        df.rename(columns={"comparison": "reconstruction"}, inplace=True)

    # Ensure 'reconstruction' exists for volume-type files
    if "reconstruction" not in df.columns:
        # some earlier code used a 3-tuple (subject, pos, level, dcte) — try to infer
        # If there is a column named 'dcte' and one named 'position', keep as-is; otherwise leave NaNs
        df["reconstruction"] = df.get("reconstruction")

    # Ensure region column exists: for volume, set WholeVolume if missing
    if "region" not in df.columns:
        df["region"] = "WholeVolume"

    # Remove WholeVolume rows from brain/eyes (these were computed elsewhere and we want tissue-level rows only)
    if mask_type in ["brain", "eyes"] and "region" in df.columns:
        df = df[df["region"] != "WholeVolume"].copy()

    # Keep only/rename expected columns and add 'mask' column
    expected_cols = ["subject", "position", "reconstruction", "region", "dcte"]
    # If some expected columns are missing, create them with NaNs so concat doesn't fail
    for c in expected_cols:
        if c not in df.columns:
            df[c] = pd.NA

    df = df[expected_cols].copy()
    df["mask"] = mask_type

    # Reorder to canonical column order
    df = df[["subject", "position", "reconstruction", "mask", "region", "dcte"]]

    dfs.append(df)

# === Merge all ===
if not dfs:
    raise FileNotFoundError("No DCTE CSVs found.")

df_all = pd.concat(dfs, ignore_index=True)

# === Ensure categorical ordering ===
df_all["mask"] = pd.Categorical(df_all["mask"], categories=["volume", "brain", "eyes"], ordered=True)
df_all["reconstruction"] = pd.Categorical(
    df_all["reconstruction"], categories=["woBin", "full", "50p", "75p", "95p"], ordered=True
)

# === Reorder columns ===
df_all = df_all[["subject", "position", "reconstruction", "mask", "region", "dcte"]]

# === Quick check ===
print("✅ Combined DataFrame shape:", df_all.shape)
print(df_all.head())
print("\nUnique mask types:", df_all["mask"].unique())
print("Unique regions:", df_all["region"].unique())
print("Unique reconstructions:", df_all["reconstruction"].unique())
print("Unique positions:", df_all["position"].unique())

# Optional: save
df_all.to_csv("dcte_all_combined.csv", index=False)
print("📁 Saved: dcte_all_combined.csv")


✅ Combined DataFrame shape: (306, 6)
   subject position reconstruction    mask       region      dcte
0  sub-001     none          woBin  volume  WholeVolume  0.967234
1  sub-001       up           full  volume  WholeVolume  1.254714
2  sub-001       up            50p  volume  WholeVolume  1.274571
3  sub-001       up            75p  volume  WholeVolume  1.086462
4  sub-001       up            95p  volume  WholeVolume  0.861575

Unique mask types: ['volume', 'brain', 'eyes']
Categories (3, object): ['volume' < 'brain' < 'eyes']
Unique regions: ['WholeVolume' 'CSF' 'GM' 'WM' 'Brain' 'Eyes']
Unique reconstructions: ['woBin', 'full', '50p', '75p', '95p']
Categories (5, object): ['woBin' < 'full' < '50p' < '75p' < '95p']
Unique positions: ['none' 'up' 'down' 'left' 'right']
📁 Saved: dcte_all_combined.csv


mean+-std

In [10]:
import pandas as pd

# === Load data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# Compute mean and std across subjects and positions
summary = df_all.groupby(['mask', 'region', 'reconstruction'])['dcte'] \
                .agg(['mean', 'std']).reset_index()

# Compute percent change relative to 'full'
full_means = summary[summary['reconstruction'] == 'full'][['mask', 'region', 'mean']]
full_means.rename(columns={'mean': 'mean_full'}, inplace=True)

# Merge to attach full reference per mask/region
summary = summary.merge(full_means, on=['mask', 'region'])
summary['perc_change'] = (summary['mean'] - summary['mean_full']) / summary['mean_full'] * 100

# Create formatted column with mean ± std (percent_change)
summary['formatted'] = summary.apply(
    lambda row: f"{row['mean']:.4f} ± {row['std']:.4f} ({row['perc_change']:+.2f}%)",
    axis=1
)

# Pivot table to have reconstructions as columns
summary_table = summary.pivot(
    index=['mask', 'region'],
    columns='reconstruction',
    values='formatted'
).reset_index()

# Optional: reorder columns if all exist
cols = ['mask', 'region', 'full', 'woBin', '50p', '75p', '95p']
summary_table = summary_table[[c for c in cols if c in summary_table.columns]]

# Add metric column and remove 'mask'
summary_table.insert(0, 'metric', 'DCTE')
if 'mask' in summary_table.columns:
    summary_table = summary_table.drop(columns=['mask'])

# Display neatly
pd.set_option('display.max_colwidth', None)

# Save table as csv
summary_table.to_csv("dcte_summary_table.csv", index=False)

summary_table

reconstruction,metric,region,full,woBin,50p,75p,95p
0,DCTE,Brain,2.8799 ± 0.3667 (+0.00%),2.4513 ± 0.3979 (-14.89%),2.6470 ± 0.2737 (-8.09%),2.3585 ± 0.2955 (-18.11%),2.0464 ± 0.1394 (-28.94%)
1,DCTE,CSF,59.3059 ± 11.1490 (+0.00%),60.2646 ± 14.8871 (+1.62%),57.0978 ± 8.8824 (-3.72%),53.2707 ± 8.0856 (-10.18%),46.1101 ± 5.1334 (-22.25%)
2,DCTE,GM,31.4408 ± 4.5040 (+0.00%),31.3747 ± 6.1674 (-0.21%),30.0414 ± 3.4997 (-4.45%),28.1593 ± 3.7550 (-10.44%),25.3483 ± 2.1086 (-19.38%)
3,DCTE,WM,12.9233 ± 1.6480 (+0.00%),12.9562 ± 2.3328 (+0.25%),12.3533 ± 1.3606 (-4.41%),11.6176 ± 1.6119 (-10.10%),10.4934 ± 0.9193 (-18.80%)
4,DCTE,Eyes,26.9859 ± 1.3069 (+0.00%),24.5426 ± 0.4433 (-9.05%),25.3900 ± 2.5384 (-5.91%),23.0798 ± 3.8462 (-14.47%),20.5647 ± 3.0389 (-23.79%)
5,DCTE,WholeVolume,1.3690 ± 0.0916 (+0.00%),1.0956 ± 0.1117 (-19.97%),1.2414 ± 0.0603 (-9.32%),1.0580 ± 0.1122 (-22.72%),0.7885 ± 0.0836 (-42.40%)


Boxplot

In [35]:
import pandas as pd
import plotly.express as px

# === Load combined DCTE data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# Ensure brain tissues are treated separately
# We'll create a combined 'mask_region' column to distinguish brain tissues
# and a nicer label 'mask_region_label' for plotting x-axis ticks for submasks

def make_mask_region(row):
    return f"{row['mask']}_{row['region']}" if row['mask'] == "brain" else row['mask']

df_all['mask_region'] = df_all.apply(make_mask_region, axis=1)

# Create human-friendly labels for mask_region (used only for the submask x-axis)
# brain_* -> use the tissue name (CSF, GM, WM, Brain), 'volume' -> 'Whole Volume', 'eyes' -> 'Eyes'

def mask_region_label(value):
    if isinstance(value, str) and value.startswith('brain_'):
        return value.split('_', 1)[1]
    if value == 'volume':
        return 'Whole Volume'
    if value == 'eyes':
        return 'Eyes'
    return value

df_all['mask_region_label'] = df_all['mask_region'].apply(mask_region_label)

# === Figure 1: DCTE vs Undersampling Level, separated by mask/region, faceted by position ===
# fig1 = px.box(
#     df_all,
#     x="reconstruction",      # undersampling levels
#     y="dcte",
#     color="mask_region",     # differentiate brain tissues + eyes/volume
#     facet_col="position",
#     points="all",
#     title="DCTE vs Undersampling Level by Mask and Region",
#     category_orders={
#         "reconstruction": ["woBin", "full", "50p", "75p", "95p"],
#         "mask_region": ["volume", "brain_CSF", "brain_GM", "brain_WM", "brain_Brain", "eyes"]
#     }
# )
# fig1.update_layout(boxmode="group")
# fig1.show()

# === Figure 2: DCTE by mask type (disentangling brain tissues), faceted by position, colored by undersampling ===
# Use the nicer mask_region_label on the x-axis so ticks read: Whole Volume, Brain, CSF, GM, WM, Eyes
fig2 = px.box(
    df_all,
    x="mask_region_label",
    y="dcte",
    color="reconstruction", # undersampling levels as color
    facet_col="position",
    points="all",
    title="DCTE by Mask Type (Brain Tissues Separated) and Position",
    category_orders={
        "mask_region_label": ["Whole Volume", "Brain", "CSF", "GM", "WM", "Eyes"],
        "reconstruction": ["woBin", "full", "50p", "75p", "95p"]
    }
)
fig2.update_layout(boxmode="group")
fig2.show()


Boxplot mean

In [42]:
import pandas as pd
import plotly.express as px

# === Load combined DCTE data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# --- Disentangle brain tissues ---
df_all['mask_region'] = df_all.apply(
    lambda row: f"{row['mask']}_{row['region']}" if row['mask'] == "brain" else row['mask'],
    axis=1
)

# === Compute mean DCTE per subject across positions ===
df_mean = df_all.groupby(
    ["subject", "mask_region", "reconstruction"], as_index=False
)["dcte"].mean()

# --- Map to simplified mask region names for ordering ---
map_simple = {
    "volume": "volume",
    "brain_Brain": "brain",
    "brain_CSF": "csf",
    "brain_GM": "gm",
    "brain_WM": "wm",
    "eyes": "eyes"
}
df_mean['mask_region_simple'] = df_mean['mask_region'].map(map_simple).fillna(df_mean['mask_region'])

# Ensure categorical ordering: volume, brain, csf, gm, wm, eyes
category_order = ["volume", "brain", "csf", "gm", "wm", "eyes"]
df_mean['mask_region_simple'] = pd.Categorical(df_mean['mask_region_simple'],
                                              categories=category_order,
                                              ordered=True)

# === Boxplot of averaged DCTE per subject ===
fig_avg = px.box(
    df_mean,
    x="mask_region_simple",
    y="dcte",
    color="reconstruction",
    points="all",
    title="Mean DCTE per Subject across Positions",
    category_orders={
        "mask_region_simple": category_order,
        "reconstruction": ["woBin", "full", "50p", "75p", "95p"]
    },
    labels={"mask_region_simple": "Mask / Brain Region"}
)
fig_avg.update_layout(boxmode="group")
fig_avg.show()


Line plot mean

In [7]:
import pandas as pd
import plotly.graph_objects as go

# === Load combined DCTE data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# (Optional) average DCTE per subject, mask/region, reconstruction if multiple positions exist
df_dcte_mean = df_all.groupby(
    ['subject', 'mask', 'region', 'reconstruction'], as_index=False
)['dcte'].mean()

# === Replace region name ===
df_dcte_mean['region'] = df_dcte_mean['region'].replace({'WholeVolume': 'Volume'})

# === Define orderings ===
recon_order = ["woBin", "full", "50p", "75p", "95p"]
region_order = ["Volume", "Brain", "CSF", "GM", "WM", "Eyes"]

df_dcte_mean['reconstruction'] = pd.Categorical(
    df_dcte_mean['reconstruction'], categories=recon_order, ordered=True
)
df_dcte_mean['region'] = pd.Categorical(
    df_dcte_mean['region'], categories=region_order, ordered=True
)
df_dcte_mean.sort_values(['region', 'reconstruction'], inplace=True)

# === Define subjects and colors ===
subjects = df_dcte_mean['subject'].unique()
color_map = {subj: color for subj, color in zip(subjects, ['red', 'blue', 'green', 'orange', 'purple'])}

# === Create combined x-axis labels ===
df_dcte_mean['x_label'] = df_dcte_mean.apply(lambda r: f"{r['reconstruction']}-{r['region']}", axis=1)

# === Plot ===
fig = go.Figure()

for subj in subjects:
    subj_df = df_dcte_mean[df_dcte_mean['subject'] == subj]
    fig.add_trace(
        go.Scatter(
            x=subj_df['x_label'],
            y=subj_df['dcte'],
            mode='lines+markers',
            name=f"Subject {subj}",
            line=dict(color=color_map[subj]),
            marker=dict(color=color_map[subj])
        )
    )

fig.update_layout(
    title="Mean DCTE per Subject across Reconstructions and Regions",
    xaxis_title="Reconstruction–Region",
    yaxis_title="DCTE",
    height=600,
    width=1200,
    template="plotly_white"
)

fig.show()


Bar chart

In [37]:
import pandas as pd
import plotly.express as px

# === Load combined DCTE data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# --- Only keep relevant regions for plotting ---
# For volume: WholeVolume
# For eyes: Eyes
# For brain: CSF, GM, WM (exclude WholeVolume)
df_all_filtered = df_all[
    ~(
        ((df_all["mask"] == "brain") & (df_all["region"] == "WholeVolume")) |
        ((df_all["mask"] == "eyes") & (df_all["region"] == "WholeVolume"))
    )
].copy()

# Ensure proper ordering
reconstruction_order = ["woBin", "full", "50p", "75p", "95p"]
position_order = ["none", "up", "down", "left", "right"]
mask_order = ["Whole Volume", "Brain", "CSF", "GM", "WM", "Eyes"]

# Build mask_region and a human-friendly label for the submask x-axis

def make_mask_region(row):
    return f"{row['mask']}_{row['region']}" if row['mask'] == "brain" else row['mask']

df_all_filtered['mask_region'] = df_all_filtered.apply(make_mask_region, axis=1)

def mask_region_label(value):
    if isinstance(value, str) and value.startswith('brain_'):
        return value.split('_', 1)[1]
    if value == 'volume':
        return 'Whole Volume'
    if value == 'eyes':
        return 'Eyes'
    return value

# Add human labels
df_all_filtered['mask_region_label'] = df_all_filtered['mask_region'].apply(mask_region_label)

# === Aggregate mean and std for DCTE ===
df_mean = (
    df_all_filtered
    .groupby(["mask", "region", "position", "reconstruction"], as_index=False)
    .agg(mean_dcte=("dcte", "mean"), std_dcte=("dcte", "std"))
)

# Use mask_region_label for plotting x-axis
# === Figure 1: reconstruction on x-axis, grouped by mask/region, faceted by position ===
# fig1 = px.bar(
#     df_mean,
#     x="reconstruction",
#     y="mean_dcte",
#     color="mask",
#     facet_col="position",
#     pattern_shape="region",  # differentiate tissues/regions visually
#     error_y="std_dcte",
#     barmode="group",
#     title="Mean DCTE ± SD across subjects (x=reconstruction)",
#     color_discrete_sequence=px.colors.qualitative.Set2,
#     category_orders={"reconstruction": reconstruction_order}
# )
# fig1.update_layout(template="plotly_white", yaxis_title="Mean DCTE", xaxis_title="Reconstruction", title_x=0.5)
# fig1.show()

# === Figure 2: mask type / region on x-axis, grouped by reconstruction, faceted by position ===
# We'll create df_mean_mask which includes the mask_region_label

df_mean['mask_region'] = df_mean.apply(lambda r: f"{r['mask']}_{r['region']}" if r['mask'] == 'brain' else r['mask'], axis=1)
ndf = df_mean.copy()
ndf['mask_region_label'] = ndf['mask_region'].apply(mask_region_label)

fig2 = px.bar(
    ndf,
    x="mask_region_label",
    y="mean_dcte",
    color="reconstruction",
    facet_col="position",
    error_y="std_dcte",
    barmode="group",
    title="Mean DCTE ± SD across subjects (x=mask/region)",
    color_discrete_sequence=px.colors.qualitative.Set2,
    category_orders={"position": position_order, "mask_region_label": mask_order, "reconstruction": reconstruction_order}
)
fig2.update_layout(template="plotly_white", yaxis_title="Mean DCTE", xaxis_title="Mask_Region", title_x=0.5)
fig2.show()


Scatter plot

In [38]:
import pandas as pd
import plotly.express as px

# === Load combined DCTE data ===
df_all = pd.read_csv("dcte_all_combined.csv")

# Disentangle brain tissues in mask_region
df_all['mask_region'] = df_all.apply(
    lambda row: f"{row['mask']}_{row['region']}" if row['mask'] == "brain" else row['mask'],
    axis=1
)

fig = px.scatter(
    df_all,
    x="reconstruction",       # undersampling levels
    y="dcte",
    color="mask_region",       # distinguish brain tissues + volume/eyes
    symbol="subject",
    facet_col="position",
    category_orders={"reconstruction": ["woBin", "full", "50p", "75p", "95p"]},
    title="DCTE per Subject and Reconstruction Type"
)

fig.update_traces(marker_size=10)
fig.update_layout(template="plotly_white")
fig.show()
